Importamos librerias necesarias

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'C:\\Users\\fpallares\Documents\\Test tecnico\\Teste Técnico - DS_\\items_titles_test.csv') #Cargamos el dataset

In [3]:
df

,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde
...,...
9995,Chuteira Futsal Oxn Velox 3 Infantil
9996,Sapatenis Casual Masculino Estiloso 24horas Co...
9997,Tênis Feminino Infantil Molekinha Tie Dye
9998,Tênis Feminino Leve Barato Ganhe 1 Colchonete ...


Importamos librerias para manipulacion de textos

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import plotly.express as px
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

#Definimos una funcion para ver las top words que mas se repiten

def get_top_n_words(corpus, n=10):
    cv = CountVectorizer() # Agregar StopWords
    corpus_matrix = cv.fit_transform(corpus)
    corpus_matrix = pd.DataFrame.sparse.from_spmatrix(corpus_matrix, columns=cv.get_feature_names_out())
    aux = corpus_matrix.sum()
    aux=aux.to_frame('count')
    aux.sort_values(by='count',ascending=False,inplace=True)
    aux=aux.head(n)
    return aux


In [6]:
#Vemos la cantidad de tokens en todo el corpus
most_importants = get_top_n_words(df["ITE_ITEM_TITLE"], n=500)
print(most_importants.head(10))
px.bar(most_importants, x=most_importants.index, y='count')
#Vemos que tenemos una gran cantidad de tokens asociados a tenis o zapatillas

           count
tênis       5750
feminino    2618
masculino   1867
tenis       1783
casual      1234
infantil    1170
preto       1123
original     880
bicicleta    745
de           583


In [7]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [8]:
# Aca definimos un patron para limpiar posibles caracteres irrelugares
non_word_pattern = r"``|''|[^A-Za-z0-9]+"

In [9]:
# Tokenizamos y hacemos el lowecase del texto
df["cleaned_text"] = df["ITE_ITEM_TITLE"].apply(lambda text: word_tokenize(text.lower()))
stop_words = set(stopwords.words("portuguese"))
df["cleaned_text"] = df["cleaned_text"].apply(lambda tokens: [word for word in tokens if word not in stop_words])

In [10]:
df

,ITE_ITEM_TITLE,cleaned_text
0,Tênis Olympikus Esporte Valente - Masculino Kids,"[tênis, olympikus, esporte, valente, -, mascul..."
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,"[bicicleta, barra, forte, samy, c/, 6, marchas..."
2,Tênis Usthemp Slip-on Temático - Labrador 2,"[tênis, usthemp, slip-on, temático, -, labrad..."
3,Tênis Casual Feminino Moleca Tecido Tie Dye,"[tênis, casual, feminino, moleca, tecido, tie,..."
4,Tênis Star Baby Sapatinho Conforto + Brinde,"[tênis, star, baby, sapatinho, conforto, +, br..."
...,...,...
9995,Chuteira Futsal Oxn Velox 3 Infantil,"[chuteira, futsal, oxn, velox, 3, infantil]"
9996,Sapatenis Casual Masculino Estiloso 24horas Co...,"[sapatenis, casual, masculino, estiloso, 24hor..."
9997,Tênis Feminino Infantil Molekinha Tie Dye,"[tênis, feminino, infantil, molekinha, tie, dye]"
9998,Tênis Feminino Leve Barato Ganhe 1 Colchonete ...,"[tênis, feminino, leve, barato, ganhe, 1, colc..."


In [11]:
df["cleaned_text"] = df["cleaned_text"].apply(lambda tokens: [word for word in tokens if not re.match(non_word_pattern, str(word))])

In [12]:
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [13]:
# Aplicamos lemmatizacion
lemmatizer = WordNetLemmatizer()
df["cleaned_text"] = df["cleaned_text"].apply(lambda tokens: [lemmatizer.lemmatize(word) for word in tokens])

In [14]:
df

,ITE_ITEM_TITLE,cleaned_text
0,Tênis Olympikus Esporte Valente - Masculino Kids,"[tênis, olympikus, esporte, valente, masculino..."
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,"[bicicleta, barra, forte, samy, c/, 6, marchas..."
2,Tênis Usthemp Slip-on Temático - Labrador 2,"[tênis, usthemp, slip-on, temático, labrador, 2]"
3,Tênis Casual Feminino Moleca Tecido Tie Dye,"[tênis, casual, feminino, moleca, tecido, tie,..."
4,Tênis Star Baby Sapatinho Conforto + Brinde,"[tênis, star, baby, sapatinho, conforto, brinde]"
...,...,...
9995,Chuteira Futsal Oxn Velox 3 Infantil,"[chuteira, futsal, oxn, velox, 3, infantil]"
9996,Sapatenis Casual Masculino Estiloso 24horas Co...,"[sapatenis, casual, masculino, estiloso, 24hor..."
9997,Tênis Feminino Infantil Molekinha Tie Dye,"[tênis, feminino, infantil, molekinha, tie, dye]"
9998,Tênis Feminino Leve Barato Ganhe 1 Colchonete ...,"[tênis, feminino, leve, barato, ganhe, 1, colc..."


In [15]:
#Joineamos los tokens en una sola cadena de char
df['joined_text'] = df['cleaned_text'].apply(lambda x: ' '.join(x))

In [16]:
df

,ITE_ITEM_TITLE,cleaned_text,joined_text
0,Tênis Olympikus Esporte Valente - Masculino Kids,"[tênis, olympikus, esporte, valente, masculino...",tênis olympikus esporte valente masculino kid
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,"[bicicleta, barra, forte, samy, c/, 6, marchas...",bicicleta barra forte samy c/ 6 marchas cubo c...
2,Tênis Usthemp Slip-on Temático - Labrador 2,"[tênis, usthemp, slip-on, temático, labrador, 2]",tênis usthemp slip-on temático labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye,"[tênis, casual, feminino, moleca, tecido, tie,...",tênis casual feminino moleca tecido tie dye
4,Tênis Star Baby Sapatinho Conforto + Brinde,"[tênis, star, baby, sapatinho, conforto, brinde]",tênis star baby sapatinho conforto brinde
...,...,...,...
9995,Chuteira Futsal Oxn Velox 3 Infantil,"[chuteira, futsal, oxn, velox, 3, infantil]",chuteira futsal oxn velox 3 infantil
9996,Sapatenis Casual Masculino Estiloso 24horas Co...,"[sapatenis, casual, masculino, estiloso, 24hor...",sapatenis casual masculino estiloso 24horas co...
9997,Tênis Feminino Infantil Molekinha Tie Dye,"[tênis, feminino, infantil, molekinha, tie, dye]",tênis feminino infantil molekinha tie dye
9998,Tênis Feminino Leve Barato Ganhe 1 Colchonete ...,"[tênis, feminino, leve, barato, ganhe, 1, colc...",tênis feminino leve barato ganhe 1 colchonete ...


In [17]:
#Aplicamos Tfidf en el texto unido
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['joined_text'])

In [18]:
df['cleaned_text_2'] = df['ITE_ITEM_TITLE'].str.split()  # split into words

In [19]:
X.shape

(10000, 8298)

A partir de esto podemos aplicar la matriz de coseno para ver que tan relacionadas estan los titulos entre si

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
cosine_matrix = cosine_similarity(X)
print(cosine_matrix)

[[1.         0.         0.01358911 ... 0.01601534 0.01110849 0.03694766]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.01358911 0.         1.         ... 0.01566584 0.01086608 0.00911359]
 ...
 [0.01601534 0.         0.01566584 ... 1.         0.04190329 0.01074074]
 [0.01110849 0.         0.01086608 ... 0.04190329 1.         0.00744995]
 [0.03694766 0.         0.00911359 ... 0.01074074 0.00744995 1.        ]]


In [22]:
import matplotlib.pyplot as plt
import seaborn as sns

Aca primero vamos a visualizar los top 5 titulos que mas match hacen, para ver si la matriz de coseno esta siendo correctamente ejecutada

In [23]:
# Para cada titulo, obtenemos los 5 titulos que mas se parecen excluyendo el primer titulo en cuestion
top_scores_indices = (-cosine_matrix).argsort()[:, 1:6]

title_list = df['ITE_ITEM_TITLE'].values # Get the list of titles

# Obtenemos los titulos mas similares
similar_titles = [[title_list[i] for i in row] for row in top_scores_indices]

# Creamos el output
df_sim = pd.DataFrame(similar_titles, index=df['ITE_ITEM_TITLE'].values, 
                      columns=[f"Similar Title {i+1}" for i in range(5)])

print(df_sim)

                                                                                      Similar Title 1  \
Tênis Olympikus Esporte Valente - Masculino Kids    Tênis Olympikus Esporte Valente - Masculino In...   
Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C/...    Bicicleta Leve Dobra Roda De Rolamento Fácil De   
Tênis Usthemp Slip-on Temático - Labrador 2              Tênis Usthemp Volare Temático - Labrador 2   
Tênis Casual Feminino Moleca Tecido Tie Dye                               Tênis Kolosh Casual Tie Dye   
Tênis Star Baby Sapatinho Conforto + Brinde         Tenis Infantil Meninas Kidy Conforto Macio Bab...   
...                                                                                               ...   
Chuteira Futsal Oxn Velox 3 Infantil                  Tênis Chuteira Oxn Velox 3 Indoor 2 Verde Limão   
Sapatenis Casual Masculino Estiloso 24horas Con...                                 Sapatênis Estiloso   
Tênis Feminino Infantil Molekinha Tie Dye              

In [24]:
df_sim.tail() #Vemos una parte del output

,Similar Title 1,Similar Title 2,Similar Title 3,Similar Title 4,Similar Title 5
Chuteira Futsal Oxn Velox 3 Infantil,Tênis Chuteira Oxn Velox 3 Indoor 2 Verde Limão,Tenis Chuteira Dostin Futsal Com Brinde Original,Chuteira Pro Soccer Futsal Infantil - Marinho ...,Chuteira Futsal Penalty Tam 36,Chuteira Dalponte Futsal Tamanho 37
Sapatenis Casual Masculino Estiloso 24horas Conforto Qualida,Sapatênis Estiloso,Tênis Masculino Casual Social Sapatenis Homem ...,Kit 2 Sapatenis 1 Tenis Casual Masculino Promo...,Tenis Sapatenis Feminino Via Marte Verão Casua...,"Kit 3 Sapatênis Masculino Casual, Confortável,..."
Tênis Feminino Infantil Molekinha Tie Dye,Tênis Kolosh Casual Tie Dye,Tênis Tie Dye Feminino + Relógio De Brinde,Tênis Casual Feminino Moleca Tecido Tie Dye,Tênis Casual Feminino Tie-dye Comfy,Tênis Infantil Feminino Bibi Roller New Tie-dye
Tênis Feminino Leve Barato Ganhe 1 Colchonete P/ Treino Azul,Tênis Barato Leve Feminino Ganhe 3 Pares Meias...,Tênis Masc. Barato Leve Fresco Ganhe 1 Par De ...,Tênis Feminino Masculino Leve Academia Caminha...,Tênis Feminino Day Confort Treino Leve Macio -...,Tênis Feminino Para Academia Azul Estiloso Pro...
Tênis Polo Ralph Lauren Modelo Cantor Low Branco Masculino,Sapatênis Polo Ralph Lauren Tamanho 41,Sapatênis De Modelo Casual Masculino Branco,Sapatenis Polo Ralph Burlap Original Masculinos,Tênis Masculino Branco,Tênis Masculino Polo Casual Confortável Promoção


In [25]:
# Necesitamos titles_list como lista para acceder al index
titles_list = df['joined_text'].tolist()

In [26]:
# Armamos vectores vacios
most_similar_titles = []
similarity_scores = []

In [27]:
#ACA ENCONTRAMOS EL MEJOR MATCH

#Aca generamos una iteracion en cada titulo a partir de cada score, y evitamos que un mismo titulo matchee como mejor titulo su mismo titulo
for idx, row in enumerate(cosine_matrix):
    row[idx] = 0  # self-similarity se seta en 0 para evitar que matchee con el su mismo titulo
    best_match_idx = row.argmax()  #Indice del mejor match
    most_similar_titles.append(titles_list[best_match_idx])  #Titulo asociado
    similarity_scores.append(row[best_match_idx])  #Score correspondiente

#Output
result_df = pd.DataFrame({
    'Title': titles_list,
    'Most Similar Title': most_similar_titles,
    'Score': similarity_scores
})


In [28]:
#Aca vemos el mejor match de un title con otro
result_df

,Title,Most Similar Title,Score
0,tênis olympikus esporte valente masculino kid,tênis olympikus esporte valente masculino inf ...,0.715786
1,bicicleta barra forte samy c/ 6 marchas cubo c...,bicicleta leve dobra roda rolamento fácil,0.259219
2,tênis usthemp slip-on temático labrador 2,tênis usthemp volare temático labrador 2,0.796797
3,tênis casual feminino moleca tecido tie dye,tênis kolosh casual tie dye,0.658061
4,tênis star baby sapatinho conforto brinde,tenis infantil meninas kidy conforto macio bab...,0.399730
...,...,...,...
9995,chuteira futsal oxn velox 3 infantil,tênis chuteira oxn velox 3 indoor 2 verde limão,0.671245
9996,sapatenis casual masculino estiloso 24horas co...,sapatênis estiloso,0.352198
9997,tênis feminino infantil molekinha tie dye,tênis kolosh casual tie dye,0.665930
9998,tênis feminino leve barato ganhe 1 colchonete ...,tênis barato leve feminino ganhe 3 pares meias...,0.452056


In [29]:
import numpy as np

In [30]:
#ACA vamos a generar el output con todas las combinaciones posibles
matches_scores = []

for idx, row in enumerate(cosine_matrix):
    row[idx] = -np.inf  # set self-comparison to -infinity to exclude from matching
    sorted_indices = np.argsort(row)[::-1]  # Ordenamos de mayor a menor los scores
    
    #Capturamos los nombres de los titulos y los score
    matches = [(titles_list[i], row[i]) for i in sorted_indices if row[i] > 0]  # Excluimos los titulos que no matchean
    
    matches_scores.append(matches)

#Output
flat_data = []
for idx, title_matches in enumerate(matches_scores):
    for match_title, score in title_matches:
        flat_data.append({
            'Title': titles_list[idx],
            'Match Title': match_title,
            'Score': score
        })


In [31]:
#Creamos el dataset y lo ordenamos nuevamente por las dudas
result_df_2 = pd.DataFrame(flat_data)
result_df_2.sort_values(by=['Score'], ascending=[False], inplace=True) 

Output

In [32]:
result_df_2 #Aca obtenemos el dataset ordenado por titulo, mejor match, y el score asociado

,Title,Match Title,Score
41344906,tenis feminino esportivo actvitta absorção imp...,tenis feminino esportivo actvitta original abs...,1.000000
22262638,sapatenis masculino polo lançamento 30 off,sapatenis masculino polo lançamento 30 off,1.000000
13771536,tenis feminino esportivo actvitta original abs...,tenis feminino esportivo actvitta absorção imp...,1.000000
42965333,sapatenis masculino polo lançamento 30 off,sapatenis masculino polo lançamento 30 off,1.000000
42872890,tênis activtta corrida masculino esportivo cat...,tênis esportivo activtta masculino cataluna co...,1.000000
...,...,...,...
33611902,tênis infantil pampili pom pom comfy tie dye 1...,tênis linha futsal branco preto marca ravia fu...,0.003012
15530422,tênis linha futsal branco preto marca ravia fu...,tênis reebok usado reebok legacy lifter menusa...,0.002995
26870959,tênis reebok usado reebok legacy lifter menusa...,tênis linha futsal branco preto marca ravia fu...,0.002995
15530423,tênis linha futsal branco preto marca ravia fu...,tênis molekinha tec thal/lyc prot/cam met cam,0.002859


In [33]:
top_1000_rows = result_df_2.head(1000)

In [34]:
top_1000_rows.to_csv('resultados_1000_caso_2.csv')